In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticate
api = KaggleApi()
api.authenticate()

# Download the dataset
dataset_name = "hoangnamnguyen91/english-vietnamese-translation"
api.dataset_download_files(dataset_name, path="/content/", unzip=True)

In [ ]:
dataset_path = "/content/en-fr.csv"